In [1]:
import scanpy as sc

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
adata = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/zebrafish_new/zebrafish_processed.h5ad")

In [3]:
adata.obs["germ_layer"].unique()

['mesoderm', 'endoderm', 'ectoderm', 'mesoderm/ectoderm', 'neural crest', 'ectoderm/neural crest', 'Neural Crest', 'mesoderm/neural crest']
Categories (8, object): ['Neural Crest', 'ectoderm', 'ectoderm/neural crest', 'endoderm', 'mesoderm', 'mesoderm/ectoderm', 'mesoderm/neural crest', 'neural crest']

In [4]:
ecto = ["ectoderm", 'neural crest', 'ectoderm/neural crest', 'Neural Crest', ]
endo = ["endoderm", ]
meso = ["mesoderm", 'mesoderm/neural crest', 'mesoderm/ectoderm']

def adapt_germ_layer(x):
    if x["germ_layer"] in ecto:
        return "ectoderm"
    if x["germ_layer"] in endo:
        return "endoderm"
    if x["germ_layer"] in meso:
        return "mesoderm"
    raise ValueError(x["germ_layer"])
    
adata.obs["germ_layer_adapted"] = adata.obs.apply(adapt_germ_layer, axis=1)

In [5]:
adata_meso = adata[(adata.obs["gene_target"]=="control_control") & (adata.obs["germ_layer_adapted"]=="mesoderm")]
adata_endo = adata[(adata.obs["gene_target"]=="control_control") & (adata.obs["germ_layer_adapted"]=="endoderm")]
adata_ecto = adata[(adata.obs["gene_target"]=="control_control") & (adata.obs["germ_layer_adapted"]=="ectoderm")]

In [6]:
sc.tl.dendrogram(adata_meso, groupby="cell_type_broad", use_rep="X_aligned")
sc.tl.dendrogram(adata_endo, groupby="cell_type_broad", use_rep="X_aligned")
sc.tl.dendrogram(adata_ecto, groupby="cell_type_broad", use_rep="X_aligned")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scanpy/tools/_dendrogram.py:193: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = dat
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scanpy/tools/_dendrogram.py:193: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = dat
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scanpy/tools/_dendrogram.py:193: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = dat


In [7]:
meso_cell_types = adata_meso.uns["dendrogram_cell_type_broad"]["categories_ordered"]
endo_cell_types = adata_endo.uns["dendrogram_cell_type_broad"]["categories_ordered"]
ecto_cell_types = adata_ecto.uns["dendrogram_cell_type_broad"]["categories_ordered"]

In [8]:
cell_types_ordered = meso_cell_types + endo_cell_types + ecto_cell_types

In [12]:
import pandas as pd
df_cts = pd.DataFrame(data=cell_types_ordered, columns=["CTs"])
df_cts["germ_layer"] = ["mesoderm"] * len(meso_cell_types) + ["endoderm"] * len(endo_cell_types) + ["ectoderm"] * len(ecto_cell_types)

In [14]:
df_cts.to_csv("/lustre/groups/ml01/workspace/ot_perturbation/data/zebrafish_new/cell_types_ordered.csv")

In [15]:
adata_pert = adata[adata.obs["gene_target"]!="control_control"]

In [16]:
meso_target = ['noto_control', 'cdx4_cdx1a', 'tbx16_msgn1', 'smo_control', 'hand2_control', 'cdx4_control', 'tbx16_tbx16l', 'tbx16_control', 'tbx1_control', 'wnt3a_wnt8', 'tbxta_control']
nervous_target = ['zc4h2_control', 'met_control', 'hgfa_control', 'epha4a_control', 'mafba_control', 'egr2b_control', 'foxi1_control', 'hoxb1a_control', 'phox2a_control']
neural_crest_target = ['tfap2a_control', 'tfap2a_foxd3', 'foxd3_control']

def get_gene_target_cat(x):
    if x["gene_target"] == "control_control":
        return "control"
    if x["gene_target"] in meso_target:
        return "mesoderm"
    if x["gene_target"] in nervous_target:
        return "nervous"
    if x["gene_target"] in neural_crest_target:
        return "neural_crest"
    raise ValueError(x["gene_target"])
    
adata.obs["gene_target_cat"] = adata.obs.apply(get_gene_target_cat, axis=1)

In [17]:
len(meso_target), len(nervous_target), len(neural_crest_target)

(11, 9, 3)

In [20]:
adata_meso_target = adata[adata.obs["gene_target_cat"]=="mesoderm"]
adata_nervous_target = adata[adata.obs["gene_target_cat"]=="nervous"]
adata_neural_crest_target = adata[adata.obs["gene_target_cat"]=="neural_crest"]

In [21]:
sc.tl.dendrogram(adata_meso_target, groupby="gene_target", use_rep="X_aligned")
sc.tl.dendrogram(adata_nervous_target, groupby="gene_target", use_rep="X_aligned")
sc.tl.dendrogram(adata_neural_crest_target, groupby="gene_target", use_rep="X_aligned")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scanpy/tools/_dendrogram.py:193: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = dat
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scanpy/tools/_dendrogram.py:193: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = dat
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scanpy/tools/_dendrogram.py:193: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = dat


In [22]:
meso_targets = adata_meso_target.uns["dendrogram_gene_target"]["categories_ordered"]
nervous_targets = adata_nervous_target.uns["dendrogram_gene_target"]["categories_ordered"]
neural_crest_targets = adata_neural_crest_target.uns["dendrogram_gene_target"]["categories_ordered"]

In [34]:
meso_targets2 = [gt + "_" + str(tp) for gt in meso_targets for tp in [18, 24, 36, 48, 72]]
nervous_targets2 = [gt + "_" + str(tp) for gt in nervous_targets for tp in [18, 24, 36, 48, 72]]
neural_crest_targets2 = [gt + "_" + str(tp) for gt in neural_crest_targets for tp in [18, 24, 36, 48, 72]]

In [36]:
gts_ordered = meso_targets2 + nervous_targets2 + neural_crest_targets2

In [37]:
df_gts = pd.DataFrame(data=gts_ordered, columns=["gene_target"])
df_gts["germ_layer"] = ["mesoderm"] * len(meso_targets2) + ["nervous"] * len(nervous_targets2) + ["neural_crest"] * len(neural_crest_targets2)

In [38]:
df_gts.to_csv("/lustre/groups/ml01/workspace/ot_perturbation/data/zebrafish_new/gene_targets_ordered.csv")

In [41]:
endo_cell_types

['liver',
 'pancreas',
 'intestine',
 'thyroid follicle cell',
 'pharynx',
 'pronephric duct',
 'swim bladder primordium']